In [1]:
from __future__ import print_function
from math import inf

# Import Python wrapper for or-tools constraint solver.
from ortools.constraint_solver import pywrapcp

def main(buffer_capacities):
  # Create the solver.
  solver = pywrapcp.Solver('jobshop')

  machines_count = 3
  buffers_count = 2
  jobs_count = 6
  all_machines = range(0, machines_count)
  all_jobs = range(0, jobs_count)
  
  # Data
  machines = [[0, 1, 2],
              [0, 1, 2],
              [0, 1, 2],
              [0, 1, 2],
              [0, 1, 2],
              [0, 1, 2]]

  processing_times = [[1, 3, 2],
                      [1, 6, 4],
                      [1, 3, 2],
                      [1, 6, 4],
                      [1, 3, 2],
                      [1, 6, 4]]

  # buffer_capacities = [3, 3]

  # Compute horizon
  horizon = 0
  for i in all_jobs:
    horizon += sum(processing_times[i])

  # Create jobs
  all_tasks = {}
  for i in all_jobs:
    for j in range(0, len(machines[i])):
      all_tasks[(i, j)] = solver.FixedDurationIntervalVar(0,
                                                          horizon,
                                                          processing_times[i][j],
                                                          False,
                                                          'Job_%i_%i' % (i, j))

  # Creates sequence variables and add disjunctive constraints.
  all_sequences = []
  all_machines_jobs = []
  for i in all_machines:

    machines_jobs = []
    for j in all_jobs:
      for k in range(0, len(machines[j])):
        if machines[j][k] == i:
          machines_jobs.append(all_tasks[(j, k)])
    disj = solver.DisjunctiveConstraint(machines_jobs, 'machine %i' % i)
    all_sequences.append(disj.SequenceVar())
    solver.Add(disj)

  # Add conjunctive contraints.
  for i in all_jobs:
    for j in range(0, len(machines[i]) - 1):
      solver.Add(all_tasks[(i, j + 1)].StartsAfterEnd(all_tasks[(i, j)]))
  
  ###
  job_present_at_enqueue = {}
  job_present_at_dequeue = {}
  # Create the Integer variables for every task's completion and starting times
  for i in range(len(all_jobs)):
    for j in range(len(all_machines) - 1):
      for k in range(len(all_jobs)):
        for l in range(len(all_machines) - 1):
          if i != k and j == l:
            # print("B_%i_%i__E_%i_%i" % (i, j, k, l))
            enq = solver.IntVar(0, 1, "B_%i_%i__E_%i_%i" % (i, j, k, l))
            job_present_at_enqueue[(i, j, k, l)] = enq
            
            # all_tasks[(p, q)].EndExpr() ---- E_pq
            # all_tasks[(p, q)].StartExpr() -- D_pq

            # enq = 1 if (E_ij <= E_kl) and (E_kl < D_ij+1)
            # AND
            # enq = 0 otherwise

            enq_value = solver.ConditionalExpression(all_tasks[(i, j)].EndExpr() <= all_tasks[(k, l)].EndExpr() and all_tasks[(k, l)].EndExpr() < all_tasks[(i, j+1)].StartExpr(), solver.IntConst(1), 0)
            solver.Add(enq == enq_value)

  for i in range(len(all_jobs)):
    for j in range(len(all_machines) - 1):
      for k in range(len(all_jobs)):
        for l in range(1, len(all_machines)):
          if i != k and j == l:
            # print("B_%i_%i__D_%i_%i" % (i, j, k, l))
            deq = solver.IntVar(0, 1, "B_%i_%i__D_%i_%i" % (i, j, k, l))
            job_present_at_dequeue[(i, j, k, l)] = deq
            
            # all_tasks[(p, q)].EndExpr() ---- E_pq
            # all_tasks[(p, q)].StartExpr() -- D_pq

            # deq = 1 if (E_ij <= D_kl) and (D_kl < D_ij+1)
            # AND
            # deq = 0 otherwise

            deq_value = solver.ConditionalExpression(all_tasks[(i, j)].EndExpr() <= all_tasks[(k, l)].StartExpr() and all_tasks[(k, l)].StartExpr() < all_tasks[(i, j+1)].StartExpr(), solver.IntConst(1), 0)
            solver.Add(deq == deq_value)
  
  for l in range(len(all_machines) - 1):
    for j in range(len(all_machines) - 1):
      for k in range(len(all_jobs)):
        sum_constraint_over = []
        for i in range(len(all_jobs)):
          if i != k and j == l:
            sum_constraint_over.append(job_present_at_enqueue[(i, j, k, l)])
        solver.Add(solver.Sum(sum_constraint_over) < buffer_capacities[j])

  # for l in range(1, len(all_machines)):
  #   for j in range(1, len(all_machines) - 1):
  #     for k in range(len(all_jobs)):
  #       sum_constraint_over = []
  #       for i in range(len(all_jobs)):
  #         if i != k and j == l:
  #           sum_constraint_over.append(job_present_at_dequeue[(i, j, k, l)])
  #       solver.Add(solver.Sum(sum_constraint_over) > 0)
  ##

  # Set the objective.
  obj_var = solver.Max([all_tasks[(i, len(machines[i])-1)].EndExpr()
                        for i in all_jobs])
  objective_monitor = solver.Minimize(obj_var, 1)
  # Create search phases.
  sequence_phase = solver.Phase([all_sequences[i] for i in all_machines],
                                solver.SEQUENCE_DEFAULT)
  vars_phase = solver.Phase([obj_var],
                            solver.CHOOSE_FIRST_UNBOUND,
                            solver.ASSIGN_MIN_VALUE)
  main_phase = solver.Compose([sequence_phase, vars_phase])
  # Create the solution collector.
  collector = solver.LastSolutionCollector()

  # Add the interesting variables to the SolutionCollector.
  collector.Add(all_sequences)
  collector.AddObjective(obj_var)

  for i in all_machines:
    sequence = all_sequences[i]
    sequence_count = sequence.Size()
    for j in range(0, sequence_count):
      t = sequence.Interval(j)
      collector.Add(t.StartExpr().Var())
      collector.Add(t.EndExpr().Var())
  # Solve the problem.
  disp_col_width = 10
  if solver.Solve(main_phase, [objective_monitor, collector]):
    print("\nOptimal Schedule Length:", collector.ObjectiveValue(0), "\n")
    sol_line = ""
    sol_line_tasks = ""
    print("Optimal Schedule", "\n")

    for i in all_machines:
      seq = all_sequences[i]
      sol_line += "Machine " + str(i) + ": "
      sol_line_tasks += "Machine " + str(i) + ": "
      sequence = collector.ForwardSequence(0, seq)
      seq_size = len(sequence)

      for j in range(0, seq_size):
        t = seq.Interval(sequence[j])
         # Add spaces to output to align columns.
        sol_line_tasks +=  t.Name() + " " * (disp_col_width - len(t.Name()))

      for j in range(0, seq_size):
        t = seq.Interval(sequence[j])
        sol_tmp = "[" + str(collector.Value(0, t.StartExpr().Var())) + ","
        sol_tmp += str(collector.Value(0, t.EndExpr().Var())) + "] "
        # Add spaces to output to align columns.
        sol_line += sol_tmp + " " * (disp_col_width - len(sol_tmp))

      sol_line += "\n"
      sol_line_tasks += "\n"

    print(sol_line_tasks)
    print("Time Intervals for Tasks\n")
    print(sol_line)
    return collector.ObjectiveValue(0)
  else:
    print('No valid solution')
    return inf

In [2]:
main([3, 3])


Optimal Schedule Length: 47 

Optimal Schedule 

Machine 0: Job_1_0   Job_0_0   Job_2_0   Job_3_0   Job_4_0   Job_5_0   
Machine 1: Job_1_1   Job_0_1   Job_2_1   Job_4_1   Job_3_1   Job_5_1   
Machine 2: Job_1_2   Job_0_2   Job_2_2   Job_4_2   Job_3_2   Job_5_2   

Time Intervals for Tasks

Machine 0: [0,1]     [1,2]     [2,3]     [3,4]     [4,5]     [5,6]     
Machine 1: [1,7]     [7,10]    [10,13]   [13,16]   [16,22]   [22,28]   
Machine 2: [7,11]    [11,13]   [13,15]   [16,18]   [22,26]   [43,47]   



47

In [4]:
import plotly.plotly as py
import plotly.graph_objs as go

X_data = []
Y_data = []
Z_data = []
for buffer_1_capacity in range(3, 7):
    for buffer_2_capacity in range(3, 7):
        optimal_sched_length = main([buffer_1_capacity, buffer_2_capacity])
        X_data.append(buffer_1_capacity)
        Y_data.append(buffer_2_capacity)
        Z_data.append(optimal_sched_length)

data = [
go.Surface(
    x=X_data,
    y=Y_data,
    z=Z_data
)
]
layout = go.Layout(
  title='Optimal schedules vs. Buffer capacities',
  autosize=True,
  margin=dict(
      l=65,
      r=50,
      b=65,
      t=90
  )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='Optimal schedules')


Optimal Schedule Length: 47 

Optimal Schedule 

Machine 0: Job_1_0   Job_0_0   Job_2_0   Job_3_0   Job_4_0   Job_5_0   
Machine 1: Job_1_1   Job_0_1   Job_2_1   Job_4_1   Job_3_1   Job_5_1   
Machine 2: Job_1_2   Job_0_2   Job_2_2   Job_4_2   Job_3_2   Job_5_2   

Time Intervals for Tasks

Machine 0: [0,1]     [1,2]     [2,3]     [3,4]     [4,5]     [5,6]     
Machine 1: [1,7]     [7,10]    [10,13]   [13,16]   [16,22]   [22,28]   
Machine 2: [7,11]    [11,13]   [13,15]   [16,18]   [22,26]   [43,47]   


Optimal Schedule Length: 47 

Optimal Schedule 

Machine 0: Job_0_0   Job_1_0   Job_2_0   Job_3_0   Job_4_0   Job_5_0   
Machine 1: Job_0_1   Job_1_1   Job_2_1   Job_4_1   Job_3_1   Job_5_1   
Machine 2: Job_0_2   Job_1_2   Job_2_2   Job_4_2   Job_3_2   Job_5_2   

Time Intervals for Tasks

Machine 0: [0,1]     [1,2]     [2,3]     [3,4]     [4,5]     [5,6]     
Machine 1: [1,4]     [4,10]    [10,13]   [13,16]   [16,22]   [22,28]   
Machine 2: [4,6]     [10,14]   [14,16]   [16,18]   [2

PlotlyError: Because you didn't supply a 'file_id' in the call, we're assuming you're trying to snag a figure from a url. You supplied the url, '', we expected it to start with 'https://plot.ly'.
Run help on this function for more information.